In [1]:
import pandas as pd
import string
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from joblib import dump, load 

# additional imports 

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import pickle

In [2]:
df=pd.read_csv('dataset.csv')

In [3]:
def text_process(mess):
    no_punc=[char for char in mess if char not in string.punctuation]
    no_punc=''.join(no_punc)
    return [word for word in no_punc.split() if word.lower() not in stopwords.words('english')]

In [4]:
pipeline=load('youarefakenews.joblib')

In [9]:
train_sample=df.sample(frac=0.2).dropna()


In [10]:
pipeline.fit(train_sample['body'],train_sample['label'])

Pipeline(memory=None,
         steps=[('bow',
                 CountVectorizer(analyzer=<function text_process at 0x7fa61f13cb00>,
                                 binary=False, decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('classifier',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [13]:
test_data=pd.read_csv('external_test_data.csv')
data=test_data.loc[test_data['language']=='english']
data=data.drop(['author', 'published', 'title', 'text', 'site_url',
       'main_img_url', 'type', 'hasImage','language'],axis=1)
data['body']=data['title_without_stopwords']+' '+data['text_without_stopwords']
data=data.drop(['title_without_stopwords','text_without_stopwords'],axis=1)
data=data.dropna()
data.head()





,label,body
0,Real,muslims busted stole millions govt benefits pr...
1,Real,attorney general loretta lynch plead fifth att...
2,Real,breaking weiner cooperating fbi hillary email ...
3,Real,pin drop speech father daughter kidnapped kill...
4,Real,fantastic trumps point plan reform healthcare ...


In [17]:
predictions=pipeline.predict(data['body'])
print(classification_report(data['label'],predictions))
print(confusion_matrix(data['label'],predictions))

              precision    recall  f1-score   support

        fake       0.63      0.83      0.72      1217
        real       0.44      0.21      0.29       750

    accuracy                           0.60      1967
   macro avg       0.54      0.52      0.50      1967
weighted avg       0.56      0.60      0.55      1967

[[1015  202]
 [ 591  159]]


In [19]:
filename = 'saved_model.sav'
pickle.dump(pipeline, open(filename, 'wb'))